In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
# from sklearn.pipeline import Pipeline
from sklearn.preprocessing import scale
from sklearn.metrics import *

import os 
os.getcwd()

'/tf'

In [4]:
data = pd.read_csv(os.getcwd() + '/data/data_reduced.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/tf/data/data_reduced.csv'

In [ ]:
data.head(20)

#preprocessing 
with data reduced not needed

In [ ]:
# TODO: CREATE A PIPELINE

# d = data
# # ratio of polymerB 
# d = d.replace({'impurityXc': {'PURE': 0}})

# #POLYFINE VS NON POLYSINE 
# d = d.replace({'Matrix_family': {'Polyolefin': 0, 'Non-Polyolefin': 1}})
# d = d.replace({'Contaminant_family': {'Polyolefin': 0, 'Non-Polyolefin': 1}})

# # Branched or linear topology 
# d = d.replace({'Matrix_topology': {'Branched': 0, 'Linear': 1}})
# d = d.replace({'Contaminant_topology': {'Branched': 0, 'Linear': 1}})

# # crystlainity, amorphous shoudl be the lowest 
# d = d.replace({'Matrix_crystallinity': {'Low': 1, 'Amorphous': 0, 'High': 2}})
# d = d.replace({'Contaminant_crystallinity': {'Low': 1, 'Amorphous': 0, 'High': 2}})
# # SCB
# d = d.replace({'Matrix_SCB': {'nan': 0, 'High': 1}})
# d = d.replace({'Contaminant_SCB': {'None': 0, 'High': 1}})

# # LCB
# d = d.replace({'Matrix_LCB': {'None': 0, 'Low' :1, 'High': 2}})
# d = d.replace({'Contaminant_LCB': {'None': 0, 'Low': 1, 'High': 2}})


# # dropping series number and name and reeks
# d = d.drop(['NAAM', 'SERIE', 'Reeks' ], axis=1)

# # theoretical estimations
# d = d.drop(['gammaABWu', 'Ddeltad2', 'Ddeltap2', 'Ddeltah2'], axis=1)
# # Kims classes 
# d = d.drop(['Matrix Class','Impurity Class', 'Class Combo', 'POL', 'POK', 'NOK', 'NOA'], axis=1) 
# # ratios 
# d = d.drop(['EblendOpEmatrix', 'STRENGTHblendOpSTRENGTHmatrix',
#        'StrainbreakblendOpStrainbreakmatrix', 'ImpactblendOpImpactmatrix' ], axis=1)

# d.columns.values

In [ ]:
# h1D = pd.get_dummies(d, columns = ['MajorityPolymer', 'MinorityPolymer'])

In [ ]:
# STANDARIZE
standardized_data = (data - data.mean())/ data.std()

# x_e = h1D.drop(['matrixSTRENGTH', 'matrixStrainbreak',
#        'matrixImpact','impuritySTRENGTH',
#        'impurityStrainbreak', 'impurityImpact', 'blendE', 'blendSTRENGTH', 'blendStrainbreak', 'blendImpact'], axis =1)
# y_e = h1D['blendE']


x_e = data.drop(['matrixSTRENGTH', 'matrixStrainbreak',
       'matrixImpact','impuritySTRENGTH',
       'impurityStrainbreak', 'impurityImpact', 'blendE', 'blendSTRENGTH', 'blendStrainbreak', 'blendImpact'], axis =1)
y_e = data['blendE']
X_train, X_test, y_train, y_test = train_test_split(x_e, y_e, test_size = 0.2, random_state = 0)

In [ ]:
param_grid = [{
            'C': [0.001, 0.01, 1, 10], 
            #   'gamma': [1, 0.75, 0.5, 0.25, 0.1, 0.01, 0.001], 
            'gamma': [1,  0.1, 0.001], 
            'kernel': ['rbf', 'poly', 'linear']
              } ]

scorings = {"NMAPE": "neg_mean_absolute_percentage_error", "NAPE": "neg_mean_absolute_error", "Var explained": "explained_variance","R2": "r2"}



svm = SVR()
grid = GridSearchCV(svm, param_grid, cv = 3, scoring = scorings, refit = 'NMAPE', return_train_score = True, verbose = 10 )
print('grid done')
grid.fit(X_train, y_train)
print('fitting done')
results = grid.cv_results_

In [ ]:
# svm.best_estimator_
# metrics.mean_absolute_percentage_error

from sklearn import metrics
# sorted(metrics.SCORERS.keys())

In [ ]:
plt.figure(figsize=(13, 13))
plt.title('GridSearch for SVR')
plt.ylable('SCORE')
plt.xlabel('hyperparam C')
ax = plt.gca()
ax.set_xlim(0.00001, 10)
ax.set_ylim(0, 1)

# Get the regular numpy array from the MaskedArray
X_axis = np.array(results["C"].data, dtype=float)
for scorer, color in zip(sorted(scorings), ["g", "k", "r", "b"]):
    for sample, style in (("train", "--"), ("test", "-")):
        sample_score_mean = results["mean_%s_%s" % (sample, scorer)]
        sample_score_std = results["std_%s_%s" % (sample, scorer)]
        ax.fill_between(
            X_axis,
            sample_score_mean - sample_score_std,
            sample_score_mean + sample_score_std,
            alpha=0.1 if sample == "test" else 0,
            color=color,
        )
        ax.plot(
            X_axis,
            sample_score_mean,
            style,
            color=color,
            alpha=1 if sample == "test" else 0.7,
            label="%s (%s)" % (scorer, sample),
        )
        
            best_index = np.nonzero(results["rank_test_%s" % scorer] == 1)[0][0]
            best_score = results["mean_test_%s" % scorer][best_index]

            # Plot a dotted vertical line at the best score for that scorer marked by x
            ax.plot(
                [
                    X_axis[best_index],
                ]
                * 2,
                [0, best_score],
                linestyle="-.",
                color=color,
                marker="x",
                markeredgewidth=3,
                ms=8,
            )

            # Annotate the best score for that scorer
            ax.annotate("%0.2f" % best_score, (X_axis[best_index], best_score + 0.005))

plt.legend(loc="best")
plt.grid(False)
plt.show()     